In [ ]:
import json
from tqdm import tqdm
from tool import soft_match_score,output_tf,filter,merge

In [ ]:
# Target数量
gold_ref = 0
pred_ref = 0
# 真正例（True Positive，TP） 正确匹配
# 硬匹配
hard_match = 0
hard_match_list = []
# 软匹配
soft_match = 0
soft_match_list = []

mathc_hateError = 0
mathc_hateError_list = []

# 假正例（False Positive，FP）预测中未在黄金标准中出现的
fp_ref = 0
fp_ref_list = []
# 假负例（False Negative，FN） 黄金标准中未在预测中出现的
fn_ref = 0
fn_ref_list = []
LOG = {}
LOG['len_gold'] = {}
LOG['len_pred'] = {}
import random
import time
# 随机种子
random.seed(time.time())
with open('../output3.json', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for i in range(len(lines)):
        lines[i] = lines[i].strip()
    # 随机访问
    lines = random.sample(lines, len(lines))
    for line in tqdm(lines):
        temp = json.loads(line)



        # if len(temp['content']) < 100:
            # continue

        if '[SEP]' not in temp['output']:
            continue




        res = temp['results'].replace(' ', '').replace('\n', '')
        res = res.replace("'指代':", "'Target':").replace("'评论':", "'Argument':").replace("'仇恨类型':", "'HateType':").replace("'是否仇恨':", "'Is':").replace("'对象':", "'Target':").replace("'是'", "'y'").replace("'否'", "'n'") 
        res = '"' + res + '"'
        try:            
            pred_quads = eval(res)
            pred_quads = eval(pred_quads)
        except:
            print('eval错误',res)
            continue
        gold_quads = output_tf(temp['output'])
        if LOG['len_gold'].get(len(gold_quads)) == None:
            LOG['len_gold'][len(gold_quads)] = 1
        else:
            LOG['len_gold'][len(gold_quads)] += 1
        if LOG['len_pred'].get(len(pred_quads)) == None:
            LOG['len_pred'][len(pred_quads)] = 1
        else:
            LOG['len_pred'][len(pred_quads)] += 1
            

        pred_quads = pred_quads[:2]
        ttt = merge(pred_quads,temp['content'])
        pred_quads = filter(pred_quads = pred_quads) 
        gold_ref += len(gold_quads)
        pred_ref += len(pred_quads)
        for i_pred in pred_quads:
            pred_Target = i_pred['Target']
            pred_Argument = i_pred['Argument']
            pred_Is = i_pred['Is']
            for index_gold in range(len(gold_quads)):
                gold_Target = gold_quads[index_gold]['Target']
                gold_Argument = gold_quads[index_gold]['Argument']
                gold_Is = gold_quads[index_gold]['Is']
                if pred_Target == gold_Target and pred_Argument == gold_Argument:
                    if pred_Is != gold_Is:
                        # if pred_Is == 'y':
                        # print(gold_quads[index_gold])
                        # print(temp['content'])
                        mathc_hateError += 1
                        mathc_hateError_list.append(i_pred)
                    else:
                        hard_match += 1
                        hard_match_list.append(i_pred)
                        gold_quads.pop(index_gold)
                    break
                elif soft_match_score(pred_Target, gold_Target, pred_Argument, gold_Argument):
                    if pred_Is != gold_Is:
                        # if pred_Is == 'y':
                        # print(gold_quads[index_gold])
                        # print(temp['content'])
                        mathc_hateError += 1
                        mathc_hateError_list.append(i_pred)
                    else:
                        soft_match += 1
                        soft_match_list.append(i_pred)
                        gold_quads.pop(index_gold)
                    break
                else:
                    fp_ref += 1
                    fp_ref_list.append(i_pred)
        for i_gold in gold_quads:
            fn_ref += 1
            fn_ref_list.append(i_gold)
        # if len(fp_ref_list) + len(fn_ref_list) > 0:
        #     print('硬匹配和软匹配========================================')
        #     print(hard_match_list)
        #     print(soft_match_list)
        #     print('假正例',fp_ref_list)
        #     print('假负例',fn_ref_list)
        #     print('mathc_hateError',mathc_hateError_list)
        #     print(output_tf(temp['output']))
        #     print(res)
        #     print(temp['content'])
        hard_match_list = []
        soft_match_list = []
        fp_ref_list = []
        fn_ref_list = []
        mathc_hateError_list = []
        

# 计算精确率、召回率和F1值
hard_precision = hard_match / (hard_match + fp_ref) if (hard_match + fp_ref) > 0 else 0
hard_recall = hard_match / (hard_match + fn_ref) if (hard_match + fn_ref) > 0 else 0
hard_f1_score = 2 * hard_precision * hard_recall / (hard_precision + hard_recall) if (hard_precision + hard_recall) > 0 else 0

soft_match_precision = soft_match / (soft_match + fp_ref) if (soft_match + fp_ref) > 0 else 0
soft_match_recall = soft_match / (soft_match + fn_ref) if (soft_match + fn_ref) > 0 else 0
soft_match_f1_score = 2 * soft_match_precision * soft_match_recall / (soft_match_precision + soft_match_recall) if (soft_match_precision + soft_match_recall) > 0 else 0
print('hard_precision:', hard_precision)
print('hard_recall:', hard_recall)
print('hard_f1_score:', hard_f1_score)
print('soft_match_precision:', soft_match_precision)
print('soft_match_recall:', soft_match_recall)
print('soft_match_f1_score:', soft_match_f1_score)
print('**f1**',(hard_f1_score+soft_match_f1_score)/2)

print(f"Hard Match: {hard_match}")
print(f"Soft Match: {soft_match}")
print(f"FP:假正例（False Positive，FP）预测中未在黄金标准中出现的", fp_ref)
print(f"FN:假负例（False Negative，FN） 黄金标准中未在预测中出现的", fn_ref)
print(f"Gold Ref: {gold_ref}")
print(f"Pred Ref: {pred_ref}")
print(f"Mathc HateError: {mathc_hateError}")
print('LOG:',LOG)